In [ ]:
from collections import Counter 
import collections
import numpy as np
import time
import os


# Local imports
from filters import *
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from scipy.spatial import distance

In [ ]:
# input function
def read_kitty(bin_file_name:str, label_file_name:str=None):
    "opens a point cloud frame following kitty formatting"
    with open(bin_file_name, mode='rb') as file:
        scan = np.fromfile(file, dtype=np.float32)
        scan = np.reshape(scan, (-1,4))
        
    if label_file_name:
        with open(label_file_name, mode='rb') as file:
            label = np.fromfile(file, dtype=np.int16)
            label = np.reshape(label, (-1,2))
        #print('l-size:',label.shape[0], 'p-size:',scan.shape[0])
        assert label.shape[0] == scan.shape[0] # assert 1 to 1 ratio
        return scan, label
    return scan

# output function
def write_kitty(folder_name:str, dataframes, labels=None, names=None):
    "writes numpy point cloud data as kitty format"
    names = names if names else [str(i) for i in range(len(dataframes))]
    if not os.path.isdir(folder_name):
        os.mkdir(os.path.join(folder_name))
        os.mkdir(os.path.join(folder_name, 'velodyne'))
        os.mkdir(os.path.join(folder_name, 'labels'))
        
    # make velodyne data
    vel_dir = os.path.join(folder_name, 'velodyne')
    for data, name in zip(dataframes, names):
        data.tofile(f'{vel_dir}/{name}.bin', sep='', format='%s')
    
    # make label data
    if labels:
        lab_dir = os.path.join(folder_name, 'labels')
        for data, name in zip(labels, names):
            data.tofile(f'{lab_dir}/{name}.label', sep='', format='%s')  

In [ ]:
from filters import *
from numpy import linalg as LA
from sklearn.metrics import precision_score
from sklearn.metrics import jaccard_score
import time

total_time=0
total_frames=0
execution_time = []


dictionary ={} #store the result of each algorithm
main_folder = 'sequences' #dataset binary labels
outlier_labels = [1] #1 :snow


for alg in ['DSOR','DROR' ,'LIOR','DSOR_and_LIOR', 'DROR_and_LIOR']:     


    y_true = None    
    y_pred = None    
    
    for inp in ['30','34','35','36','76' ]: #test data

        # Get all datapaths
        Input_folder = os.path.join(main_folder, inp)
        scan_path = os.path.join(Input_folder, 'velodyne')
        label_path = os.path.join(Input_folder, 'labels')


        scan_names = sorted([os.path.join(dp, f) 
                  for dp, dn, fn in os.walk(os.path.expanduser(scan_path)) 
                  for f in fn])

        label_names = sorted([os.path.join(dp, f) 
                   for dp, dn, fn in os.walk(os.path.expanduser(label_path)) 
                   for f in fn])

        #assert all files have corresponding labels
        scan_path_len = len(scan_path)+1
        label_path_len = len(label_path)+1
        names = [scan[scan_path_len:-4] for scan in scan_names]
        assert all(name == label[label_path_len:-6] for name, label in zip(names, label_names))

        
        for scan, label in zip(scan_names, label_names):
           

            points, labels = read_kitty(scan, label)  #fetch the data point cloud 
            binary_labels = np.isin(labels[:,0], outlier_labels) # Labels 
                
            end = time.time()
            if 'DSOR_and_LIOR' == alg:
                end = time.time()
                mask = DSOR_and_LIOR(points, 0.05, 4, 288, 0.066, Sfactor=0.04, r=0.08)
            elif 'DROR' == alg:
                end = time.time()
                mask = DROR(points, 0.2, 0.16, 7, 0.08, 20)
            elif 'DSOR' == alg:
                end = time.time()
                mask = DSOR(points, 0.04, 0.08, 4) 
            elif 'DROR_and_LIOR' == alg:
                end = time.time()
                mask = DROR_and_LIOR(points, 0.05, 7, 144, 0.066,0.2, 0.16, 0.08, 20 )
            elif 'LIOR' == alg:
                end = time.time()
                mask  = LIOR(points,0.05, 2, 144, 0.066)

            frame_time = time.time() - end 
            total_frames += 1 
            execution_time.append(frame_time)
            '''  '''
            if y_true is None:
                y_true = labels[:,0]
                y_pred = mask
            else:   
                y_true = np.concatenate((y_true, labels[:,0]))
                y_pred = np.concatenate((y_pred, mask))
     
    print("Mean time:{}\t std:{}".format(np.mean(execution_time)))  
    
    print('filter: ',alg)
    dictionary[alg] = f'Precision:{precision_score(y_true, y_pred, average=None)[1]:.5f}\
                      , Recall:{recall_score(y_true, y_pred, average=None)[1]:.5f}\
                      , F1-score:{(f1_score(y_true, y_pred , average=None)[1]):.5f}\
                        '                    
'Done!' 
dictionary